In [ ]:
# Import necessary libraries
import pandas as pd
from requests import Request, Session
import json
import os
from time import sleep

In [ ]:
# Define the API endpoint and parameters
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
    'start': '1',
    'limit': '15',  # Updated to focus on the top 15 cryptocurrencies instead of the initial 5000
    'convert': 'USD'
}
# Set the API key for authentication
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': '3dc9f005-19a0-456b-ad8f-28b85f789732',
}

# Create a session and set headers
session = Session()
session.headers.update(headers)

try:
    # Send a GET request to the API endpoint with specified parameters
    response = session.get(url, params=parameters)
    data = json.loads(response.text)  # Parse the JSON response
    print(data)  # Print the JSON data
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)  # Handle connection errors
else:
    # Save the JSON data to a file named 'data.json', indenting for readability
    with open("data.json", 'w') as json_file:
        json.dump(data, json_file, indent=2)  # Save the data into a JSON file and indent it for readability


In [ ]:
# Set display options for pandas DataFrame
# This allows you to see all the columns and rows, not just like 15
pd.set_option('display.max_columns', None,'display.max_rows', None)
# Set display options for float format
pd.set_option('display.float_format', '{:.5f}'.format)

In [ ]:
# Normalize JSON data and create a DataFrame
dframe = pd.json_normalize(data['data'])
dframe

## SECOND VIDEO

In [ ]:
# Define column names for the DataFrame
columns = [
    "id", "name", "symbol", "slug", "num_market_pairs", "date_added", "tags",
    "max_supply", "circulating_supply", "total_supply", "infinite_supply",
    "platform", "cmc_rank", "self_reported_circulating_supply",
    "self_reported_market_cap", "tvl_ratio", "last_updated",
    "quote.USD.price", "quote.USD.volume_24h", "quote.USD.volume_change_24h",
    "quote.USD.percent_change_1h", "quote.USD.percent_change_24h",
    "quote.USD.percent_change_7d", "quote.USD.percent_change_30d",
    "quote.USD.percent_change_60d", "quote.USD.percent_change_90d",
    "quote.USD.market_cap", "quote.USD.market_cap_dominance",
    "quote.USD.fully_diluted_market_cap", "quote.USD.tvl",
    "quote.USD.last_updated", "platform.id", "platform.name",
    "platform.symbol", "platform.slug", "platform.token_address", "timestamp"
]

# Create an empty DataFrame with specified columns
dframe = pd.DataFrame(columns=columns)

In [ ]:
# Function to fetch data from the API and save to CSV
def api_runner():
    global dframe
    # Same API endpoint and parameters as before
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
        'start': '1',
        'limit': '15',  # Updated to focus on the top 15 cryptocurrencies instead of the initial 5000
        'convert': 'USD'
    }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': '3dc9f005-19a0-456b-ad8f-28b85f789732',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
    else:
        dframe2 = pd.json_normalize(data['data'])
        dframe2['timestamp'] = pd.to_datetime('now', utc=True)
        dframe = pd.concat([dframe, dframe2], ignore_index=False)
        save_to_csv(dframe2)  # Save the new data to CSV


In [ ]:
# Function to save DataFrame to CSV file
def save_to_csv(df):
    # Specify the file path
    file_path = r"C:\Users\ORESANYA\Classic Isaac\Python\CoinMarketData.csv"
    #file_path = r"C:\path\to\CoinMarketData.csv"

    # Check if the file exists
    if not os.path.isfile(file_path):
        # If the file does not exist, write headers
        df.to_csv(file_path, header="column_names", index=False)
    else:
        # If the file exists, append data without headers
        df.to_csv(file_path, mode='a', header=False, index=False)


In [ ]:
# Main function to run API calls and save data
def main():
    for i in range(5):
        api_runner()  # Call the API runner function
        print('API Runner completed')
        sleep(60)  # Sleep for 1 minute before next API call
    exit()

In [ ]:
main()

In [ ]:
# Read data from CSV file
dframe3 = pd.read_csv(r"C:\Users\ORESANYA\Classic Isaac\Python\CoinMarketData.csv")
dframe3


In [ ]:
# Group data by cryptocurrency name and calculate mean percent changes
dframe4 = dframe3.groupby('name', sort=False)[[
    'quote.USD.percent_change_1h',
    'quote.USD.percent_change_24h',
    'quote.USD.percent_change_7d',
    'quote.USD.percent_change_30d',
    'quote.USD.percent_change_60d',
    'quote.USD.percent_change_90d']].mean()

dframe4

In [ ]:
# Stack the DataFrame to create a multi-level index
dframe5 = dframe4.stack()
dframe5

In [ ]:
print(type(dframe5))

# Convert the stacked data to a DataFrame
dframe6 = dframe5.to_frame(name='values')

# Reset index and rename columns
dframe6.reset_index(inplace=True)
dframe6.rename(columns={'name': 'cryptocurrency', 'level_1': 'percent_change'}, inplace=True)
dframe6 